# Eye of Emergency 🚨 - Analyse des Tweets de Catastrophes

## Objectif du projet
Développement d'un modèle d'apprentissage automatique capable de classer des tweets signalant des catastrophes naturelles réelles pour aider les intervenants d'urgence et le public à accéder à des informations précises et fiables en période de crise.



In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [7]:
df = pd.read_csv('../data/raw/train_tweets.csv', index_col='id')


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7613 entries, 1 to 10873
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   keyword   7552 non-null   object
 1   location  5080 non-null   object
 2   text      7613 non-null   object
 3   target    7613 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 297.4+ KB


In [8]:
df.head()

,keyword,location,text,target
id,,,,
1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [10]:
df.describe(include='all')

,keyword,location,text,target
count,7552,5080,7613,7613.00000
unique,221,3341,7503,NaN
top,fatalities,USA,11-Year-Old Boy Charged With Manslaughter of T...,NaN
freq,45,104,10,NaN
mean,NaN,NaN,NaN,0.42966
std,NaN,NaN,NaN,0.49506
min,NaN,NaN,NaN,0.00000
25%,NaN,NaN,NaN,0.00000
50%,NaN,NaN,NaN,0.00000
75%,NaN,NaN,NaN,1.00000


### Remarque
- Les id ne sont pas continus : de 1 à 10873<br>
Alors qu'il y a 7613 lignes<br>
- `keyword` et surtout `location` ont des valeurs manquantes (respectivement 61 et 2533)

In [ ]:
df.isna().sum()

keyword       61
location    2533
text           0
target         0
dtype: int64

### `target`
Le jeu de donné est assez équilibré
57 / 43 %

In [41]:
target_distrib = df['target'].value_counts()
# Calculer le pourcentage de chaque classe
target_normalized = df['target'].value_counts(normalize=True) * 100
print(f"Distribution des classes : {target_distrib}\n\n\
      Pourcentage de chaque classe :\n{target_normalized}")

Distribution des classes : target
0    4342
1    3271
Name: count, dtype: int64

      Pourcentage de chaque classe :
target
0    57.034021
1    42.965979
Name: proportion, dtype: float64


### `keyword`

In [35]:
print(f"{df['keyword'].value_counts()}\n\n\
    Nombre de keyword : {df['keyword'].value_counts().sum()}\n\n\
    Nombre de keyword uniques : {df['keyword'].nunique()}")

keyword
fatalities               45
deluge                   42
armageddon               42
sinking                  41
damage                   41
                         ..
forest%20fire            19
epicentre                12
threat                   11
inundation               10
radiation%20emergency     9
Name: count, Length: 221, dtype: int64

    Nombre de keyword : 7552

    Nombre de keyword uniques : 221


### `localisations`
Il faudra les normaliser

In [36]:
print(f"{df['location'].value_counts()}\n\n\
    Nombre de localisations : {df['location'].value_counts().sum()}\n\n\
    Nombre de localisations uniques : {df['location'].nunique()}")

location
USA                    104
New York                71
United States           50
London                  45
Canada                  29
                      ... 
MontrÌ©al, QuÌ©bec       1
Montreal                 1
ÌÏT: 6.4682,3.18287      1
Live4Heed??              1
Lincoln                  1
Name: count, Length: 3341, dtype: int64

    Nombre de localisations : 5080

    Nombre de localisations uniques : 3341


### `text`

In [46]:
print(f"Nombre de textes : {df['text'].value_counts().sum()}\n\n\
Nombre de textes uniques : {df['text'].nunique()}")

Nombre de textes : 7613

Nombre de textes uniques : 7503


In [49]:
# Identifie les textes dupliqués
duplicated_texts = df['text'][df['text'].duplicated(keep=False)]

# Affiche les lignes correspondantes
print(df[df['text'].isin(duplicated_texts)])


          keyword        location  \
id                                  
59         ablaze  Live On Webcam   
68         ablaze  Live On Webcam   
156    aftershock              US   
165    aftershock              US   
171    aftershock     Switzerland   
...           ...             ...   
10855         NaN             NaN   
10867         NaN             NaN   
10870         NaN             NaN   
10871         NaN             NaN   
10872         NaN             NaN   

                                                    text  target  
id                                                                
59     Check these out: http://t.co/rOI2NSmEJJ http:/...       0  
68     Check these out: http://t.co/rOI2NSmEJJ http:/...       0  
156    320 [IR] ICEMOON [AFTERSHOCK] | http://t.co/vA...       0  
165    320 [IR] ICEMOON [AFTERSHOCK] | http://t.co/vA...       0  
171    320 [IR] ICEMOON [AFTERSHOCK] | http://t.co/TH...       0  
...                                              

In [56]:
#  Détection des doublons
df[df['text'].duplicated(keep=False)]


,keyword,location,text,target
id,,,,
59,ablaze,Live On Webcam,Check these out: http://t.co/rOI2NSmEJJ http:/...,0
68,ablaze,Live On Webcam,Check these out: http://t.co/rOI2NSmEJJ http:/...,0
156,aftershock,US,320 [IR] ICEMOON [AFTERSHOCK] | http://t.co/vA...,0
165,aftershock,US,320 [IR] ICEMOON [AFTERSHOCK] | http://t.co/vA...,0
171,aftershock,Switzerland,320 [IR] ICEMOON [AFTERSHOCK] | http://t.co/TH...,0
...,...,...,...,...
10855,NaN,NaN,Evacuation order lifted for town of Roosevelt:...,1
10867,NaN,NaN,#stormchase Violent Record Breaking EF-5 El Re...,1
10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1


In [60]:
print(f"Nombre de doublons complets : {df.duplicated().sum()}")

Nombre de doublons complets : 52


In [61]:
print(f"Nombre de tweets dupliqués : {df['text'].duplicated().sum()}")


Nombre de tweets dupliqués : 110


In [ ]:
# Compte les doublons de texte
df['text'].value_counts()[lambda x: x > 1].sum()

179